In [1]:
import datetime
import pandas as pd
import reverse_geocoder as rg 
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Geocoder")
import plotly.graph_objects as go
from google.cloud import bigquery
from datetime import date,timedelta

In [2]:
today = date.today()
yesterday = today - timedelta(days = 1) 
date_format = yesterday.strftime("%m-%d-%Y")

sql = """
    SELECT *
    FROM `not-a-doctor-273222.Topic_Modelling.CovidData`
    """ 

In [3]:
project_id = 'not-a-doctor-273222'
data = pd.read_gbq(sql, project_id=project_id, dialect='standard')
data.head()

Downloading: 100%|███████████████████████████████████████████████████████████| 98669/98669 [00:10<00:00, 9004.68rows/s]


,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Date_uploaded
0,New York City,New York,US,3/22/20 23:45,40.767273,-73.971526,9654,63,0,0.0,"New York City, New York, US",03-22-2020
1,Nassau,New York,US,3/22/20 23:45,40.740665,-73.589419,1900,4,0,0.0,"Nassau, New York, US",03-22-2020
2,Suffolk,New York,US,3/22/20 23:45,40.883201,-72.801217,1034,9,0,0.0,"Suffolk, New York, US",03-22-2020
3,King,Washington,US,3/22/20 23:45,47.491379,-121.834613,1040,75,0,0.0,"King, Washington, US",03-22-2020
4,Snohomish,Washington,US,3/22/20 23:45,48.046160,-121.717070,480,10,0,0.0,"Snohomish, Washington, US",03-22-2020


In [4]:
trend_data = data.groupby(['Date_uploaded']).sum()

In [6]:
index = trend_data.index
a_list = list(index)
print(a_list)
trend_data['date'] = a_list

['01-22-2020', '01-23-2020', '01-24-2020', '01-25-2020', '01-26-2020', '01-27-2020', '01-28-2020', '01-29-2020', '01-30-2020', '01-31-2020', '02-01-2020', '02-02-2020', '02-03-2020', '02-04-2020', '02-05-2020', '02-06-2020', '02-07-2020', '02-08-2020', '02-09-2020', '02-10-2020', '02-11-2020', '02-12-2020', '02-13-2020', '02-14-2020', '02-15-2020', '02-16-2020', '02-17-2020', '02-18-2020', '02-19-2020', '02-20-2020', '02-21-2020', '02-22-2020', '02-23-2020', '02-24-2020', '02-25-2020', '02-26-2020', '02-27-2020', '02-28-2020', '02-29-2020', '03-01-2020', '03-02-2020', '03-03-2020', '03-04-2020', '03-05-2020', '03-06-2020', '03-07-2020', '03-08-2020', '03-09-2020', '03-10-2020', '03-11-2020', '03-12-2020', '03-13-2020', '03-14-2020', '03-15-2020', '03-16-2020', '03-17-2020', '03-18-2020', '03-19-2020', '03-20-2020', '03-21-2020', '03-22-2020', '03-23-2020', '03-24-2020', '03-25-2020', '03-26-2020', '03-27-2020', '03-28-2020', '03-29-2020', '03-30-2020', '03-31-2020', '04-01-2020', '04-0

In [7]:
x = trend_data['date']
y = trend_data['Confirmed']

In [8]:
trend_data['date']= pd.to_datetime(trend_data['date']) 

In [11]:
template_cases = "<b>%{y}</b> confirmed cases on <extra></extra>"
template_deaths = "<b>%{y}</b> deaths on <extra></extra>"
template_recovered = "<b>%{y}</b> recovered on <extra></extra>"

fig = go.Figure()
fig.add_trace(
            go.Scatter(
            x=trend_data["date"],
            y=trend_data["Confirmed"],
            name="Confirmed Cases",
            line={"color": "#F4B000"},
            mode='lines+markers',
            hovertemplate=template_cases+'<b>%{text}</b>',
                text = trend_data['date'].dt.strftime('%d %b %y')
        )
    )

fig.add_trace(
            go.Scatter(
            x=trend_data["date"],
            y=trend_data["Deaths"],
            name="Deaths",
            line={"color": "#870000"},
            mode="lines+markers",
            hovertemplate=template_deaths+'<b>%{text}</b>',
                text = trend_data['date'].dt.strftime('%d %b %y')
        ),
    )
fig.add_trace(
            go.Scatter(
            x=trend_data["date"],
            y=trend_data["Recovered"],
            name="Recovered",
            line={"color": "#008000"},
            mode="lines+markers",
            hovertemplate=template_recovered+'<b>%{text}</b>',
                text = trend_data['date'].dt.strftime('%d %b %y')
        ),
    )
fig.update_layout(
        title={
        'text': "COVID worldwide trend over time",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font':dict(family="Roboto, sans-serif", size=20, color="#f4f4f4")},
#         title="COVID trends over time",
#         xaxis_title="Date",
#         yaxis_title="Population",
        #margin={"r": 0, "t": 0, "l": 0, "b": 1},
        template="plotly_dark",
        # annotations=annotations,
        autosize=True,
        showlegend=True,
        legend_orientation="v",
        #paper_bgcolor="rgba(0,0,0,0)",
        #plot_bgcolor="rgba(0,0,0,0)",
        # xaxis_title="Number of Days",
        yaxis={"linecolor": "rgba(0,0,0,0)"},
        hoverlabel={"font": {"color": "black"}},
        xaxis_showgrid=False,
        yaxis_showgrid=True,
        xaxis={"tickformat":"%b","nticks":5},
        font=dict(family="Roboto, sans-serif", size=10, color="#f4f4f4"),
        legend=dict(
            title=None, orientation="h", y=-0.3, yanchor="bottom", x=0, xanchor="left"
        ),
    )
fig.show()